In [14]:
# Convert RTTM files to UEM
# Also createas a LST file
# Reference: https://github.com/nryant/dscore#uem

dataPath = "/home/jovyan/work/data/voxconverse/sample"
rttmPath = "/home/jovyan/work/lists/voxconverse/sample.rttm"
lstPath = "/home/jovyan/work/lists/voxconverse/sample.lst"
uemPath = "/home/jovyan/work/lists/voxconverse/sample.uem"

In [63]:
rttmLines = []
with open(rttmPath, 'r') as file:
    for line in file.readlines():
        rttmLines.append(line.strip())

In [68]:
fileIds = []
fileInfos = []

for idx,line in enumerate(rttmLines):
    rttmInfo = line.split()
    fileId = rttmInfo[1]
    
    if fileId in fileIds:
        continue

    fileInfo = {
        "fileId": fileId,
        "firstTurnIndex": idx,
        "turnOnset": rttmInfo[3],
        "turnDuration": rttmInfo[4]
    }
    fileIds.append(fileId)
    fileInfos.append(fileInfo)    

In [69]:
uemLines=[]
lastFileIndex = len(fileInfos) - 1

for fileIdx, info in enumerate(fileInfos):
    firstTurnIndex= info["firstTurnIndex"]
    firstTurnOnset = info["turnOnset"]
    lastTurnOnset = 0
    lastTurnDuration = 0

    if fileIdx == lastFileIndex:
        lastTurnInfos = rttmLines[-1].split()
        lastTurnOnset = lastTurnInfos[3]
        lastTurnDuration = lastTurnInfos[4]
    else:
        nextFileInfos = fileInfos[int(fileIdx)+1]        
        nextFileFirstTurnIndex = nextFileInfos['firstTurnIndex']

        lastTurnInfos = rttmLines[nextFileFirstTurnIndex-1].split()
        lastTurnOnset = lastTurnInfos[3]
        lastTurnDuration = lastTurnInfos[4]


    fileId = info['fileId']
    channelId = 1
    onset = firstTurnOnset
    offset = float(lastTurnOnset) + float(lastTurnDuration)

    uemLine = "{} {} {} {}".format(fileId, channelId, onset, round(offset, 2))
    uemLines.append(uemLine)

In [70]:
with open(lstPath, "w") as lstFile:
    lstFile.write("\n".join(fileIds))

with open(uemPath, "w") as uemFile:
    uemFile.write("\n".join(uemLines))